In [1]:
from allennlp.data.dataset_readers import DatasetReader, Seq2SeqDatasetReader
from allennlp.data.tokenizers import Tokenizer, WordTokenizer, CharacterTokenizer
from allennlp.data.fields import LabelField, TextField, Field
from allennlp.data import Instance
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer, TokenCharactersIndexer
from allennlp.data.iterators import BucketIterator

from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder

# preprocessing 
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

# encoder
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.modules.seq2seq_encoders import StackedSelfAttentionEncoder

# model
from allennlp.nn.activations import Activation
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.training.trainer import Trainer
from allennlp.data.iterators import BucketIterator

# attention
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention

# other packages
from overrides import overrides
import pandas as pd
from collections import namedtuple
from sklearn.model_selection import train_test_split
import itertools
from allennlp.predictors import SimpleSeq2SeqPredictor

# pytorch
import torch
import torch.nn as nn
import torch.optim as optim




In [2]:
torch.cuda.is_available()

True

In [3]:
Config = namedtuple('Config', [
                    'lazy',
                    'max_vocab_size', 
                    'batch_size', 
                    'lr', 
                    'epochs', 
                    'max_seq_len', 
                    'IN_EMBEDDING_DIM', 
                    'HIDDEN_DIM', 
                    'OUT_EMBEDDING_DIM', 
                    'CUDA_DEVICE'
])

In [4]:
config = Config(False,
               10000,
               32,
               10e-4,
               10,
               128,
               64,
               32,
               32,
               0 if torch.cuda.is_available() else -1)

In [5]:
reader = Seq2SeqDatasetReader(
    source_tokenizer = CharacterTokenizer(),
    target_tokenizer = CharacterTokenizer(),
    source_token_indexers={'tokens': SingleIdTokenIndexer()},
    target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')},
    lazy=config.lazy
)

In [ ]:
raw_dataset = pd.read_csv('data/ru_train.csv')

In [ ]:
raw_dataset['before'] = raw_dataset['before'].astype(str)
raw_dataset['after'] = raw_dataset['after'].astype(str)

In [ ]:
df = raw_dataset[:].groupby('sentence_id').agg({
    'before': lambda x: ' '.join(x),
    'after': lambda x: ' '.join(x)
})

In [ ]:
df_train, df_test = train_test_split(df)

In [ ]:
df_train[:100000].to_csv('train_dataset.tsv', index=False, header=False, sep='\t')
df_test[:1000].to_csv('test_dataset.tsv', index=False, header=False, sep='\t')

In [6]:
train_dataset = reader.read('train_dataset.tsv',)
validation_dataset = reader.read('test_dataset.tsv')

100000it [00:28, 3541.26it/s]
1000it [00:00, 6953.80it/s]


# prepare vocabulary

In [7]:
vocab = Vocabulary.from_instances(train_dataset,
                                  min_count={'tokens': 3, 'target_tokens': 3}
                                 )

100%|██████████| 100000/100000 [00:09<00:00, 10342.41it/s]


In [8]:
vocab_test = Vocabulary.from_instances(validation_dataset,
                                       min_count={'tokens': 3, 'target_tokens': 3}
                                      )

100%|██████████| 1000/1000 [00:00<00:00, 9661.02it/s]


In [9]:
in_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=config.IN_EMBEDDING_DIM
                        )

In [10]:
source_embedder = BasicTextFieldEmbedder({"tokens": in_embedding})

In [11]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("source_tokens", "num_tokens")],
                         )

In [12]:
iterator.index_with(vocab)

# prepare model

In [14]:
## get attention encoder

encoder = StackedSelfAttentionEncoder(input_dim=config.IN_EMBEDDING_DIM, 
                                      hidden_dim=config.HIDDEN_DIM, 
                                      projection_dim=64, 
                                      feedforward_hidden_dim=64, 
                                      num_layers=1, 
                                      num_attention_heads=8)

attention = DotProductAttention()

In [15]:
max_decoding_steps = 100

model = SimpleSeq2Seq(vocab, 
                      source_embedder, 
                      encoder, 
                      max_decoding_steps,
                      target_embedding_dim=config.OUT_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      use_bleu=True,
                      attention=attention,
                      scheduled_sampling_ratio = 0.15)

In [16]:
if torch.cuda.is_available():
    model.cuda(config.CUDA_DEVICE)

In [17]:
model

SimpleSeq2Seq(
  (_source_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_encoder): StackedSelfAttentionEncoder(
    (feedforward_0): FeedForward(
      (_linear_layers): ModuleList(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Linear(in_features=64, out_features=32, bias=True)
      )
      (_dropout): ModuleList(
        (0): Dropout(p=0.1, inplace=False)
        (1): Dropout(p=0.1, inplace=False)
      )
    )
    (feedforward_layer_norm_0): LayerNorm()
    (self_attention_0): MultiHeadSelfAttention(
      (_combined_projection): Linear(in_features=32, out_features=192, bias=True)
      (_output_projection): Linear(in_features=64, out_features=32, bias=True)
      (_attention_dropout): Dropout(p=0.1, inplace=False)
    )
    (layer_norm_0): LayerNorm()
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (_attention): DotProductAttention()
  (_target_embedder): Embedding()
  (_decoder_cell): LSTMCell(64, 32)
  (_outpu

In [18]:
optimizer = optim.Adam(model.parameters())

In [19]:
# training

In [20]:
torch.cuda.is_available()

True

In [21]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  num_epochs=1,
                  cuda_device=config.CUDA_DEVICE)

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


In [22]:
print(f'Will train for {config.epochs} epochs')
for i in range(config.epochs):
    print(f'Epoch: {i+1}')
    trainer.train()

    predictor = SimpleSeq2SeqPredictor(model, reader)

    for instance in itertools.islice(validation_dataset, 0, 100, 10):
        print('SOURCE:', instance.fields['source_tokens'].tokens)
        print('GOLD:', instance.fields['target_tokens'].tokens)
        print('PRED:', predictor.predict_instance(instance)['predicted_tokens'])

  0%|          | 0/3125 [00:00<?, ?it/s]

Will train for 10 epochs
Epoch: 1


loss: 3.5821 ||:  17%|█▋        | 521/3125 [03:47<18:57,  2.29it/s] 


RuntimeError: CUDA out of memory. Tried to allocate 320.00 MiB (GPU 0; 5.94 GiB total capacity; 5.11 GiB already allocated; 7.81 MiB free; 5.13 GiB reserved in total by PyTorch)